In [1]:
import pandas as pd
import plotly.express as px
import os

# Configuración
pd.options.plotting.backend = "plotly"

# Cargar datos
FILE_2000_2011 = '../data/processed/fondef_2000_2011_processed.csv'
FILE_2012_2017 = '../data/processed/fondef_2012_2017_processed.csv'

df_2000 = pd.read_csv(FILE_2000_2011)
df_2012 = pd.read_csv(FILE_2012_2017)

# Unificar dataset para conteos
df_all = pd.concat([df_2000, df_2012], ignore_index=True)
# Limpiar años inválidos
df_all = df_all[df_all['year'] > 1990]

print(f"Total de proyectos analizados: {len(df_all)}")

Total de proyectos analizados: 1099


## 1. ¿Qué áreas tienen más proyectos? (2000-2017)

Un **gráfico de barras horizontales** es ideal para comparar categorías cuando los nombres son largos.

In [2]:
# Conteo por área
count_by_area = df_all['area'].value_counts().reset_index()
count_by_area.columns = ['Área', 'Cantidad de Proyectos']

fig1 = px.bar(count_by_area, 
              x='Cantidad de Proyectos', 
              y='Área', 
              orientation='h',
              title='Total de Proyectos Adjudicados por Área (2000-2017)',
              text='Cantidad de Proyectos',
              color='Cantidad de Proyectos',
              color_continuous_scale='Blues')

fig1.update_layout(yaxis={'categoryorder':'total ascending'})
fig1.show()

## 2. ¿Cómo ha evolucionado la adjudicación de proyectos?

Un **gráfico de líneas** es el estándar para mostrar tendencias a lo largo del tiempo.

In [3]:
# Conteo por año
projects_per_year = df_all.groupby('year').size().reset_index(name='Cantidad')

fig2 = px.line(projects_per_year, 
               x='year', 
               y='Cantidad', 
               title='Evolución de la Cantidad de Proyectos por Año',
               markers=True,
               labels={'year': 'Año', 'Cantidad': 'N° de Proyectos'})

fig2.update_traces(line_color='#E63946', line_width=3)
fig2.update_layout(xaxis_tickmode='linear')
fig2.show()

## 3. Distribución del Presupuesto (2000-2011)

Un **gráfico de torta (Pie Chart)** nos permite ver rápidamente la "tajada" que se lleva cada área del presupuesto total disponible en el periodo donde tenemos datos de montos.

In [4]:
# Filtrar datos con montos
df_amounts = df_2000[df_2000['amount'] > 0]
amount_by_area = df_amounts.groupby('area')['amount'].sum().reset_index()

fig3 = px.pie(amount_by_area, 
              values='amount', 
              names='area', 
              title='Distribución del Presupuesto por Área (2000-2011)',
              hole=0.3) # Gráfico de dona para un look más moderno

fig3.update_traces(textposition='inside', textinfo='percent+label')
fig3.show()

## 4. Top 5 Áreas: Evolución Comparativa

Un **gráfico de barras agrupadas** para ver cómo compiten las áreas principales año a año.

In [5]:
# Identificar Top 5 áreas
top_5_areas = df_all['area'].value_counts().head(5).index.tolist()

# Filtrar dataset
df_top5 = df_all[df_all['area'].isin(top_5_areas)]

# Agrupar
evolution_top5 = df_top5.groupby(['year', 'area']).size().reset_index(name='Cantidad')

fig4 = px.bar(evolution_top5, 
              x='year', 
              y='Cantidad', 
              color='area', 
              barmode='group',
              title='Evolución de las 5 Áreas Principales',
              labels={'year': 'Año', 'Cantidad': 'N° Proyectos'})

fig4.update_layout(xaxis_tickmode='linear')
fig4.show()

In [6]:
# Agrupar montos por año y área
amount_evolution = df_amounts.groupby(['year', 'area'])['amount'].sum().reset_index()

fig5 = px.bar(amount_evolution, 
              x='year', 
              y='amount', 
              color='area', 
              title='Evolución de Montos por Área (2000-2011)',
              labels={'year': 'Año', 'amount': 'Monto (Miles $)'},
              text_auto='.2s')

fig5.update_layout(xaxis_tickmode='linear')
fig5.show()

## 5. Montos por Área y Año (2000-2011)

Un **gráfico de barras apiladas** para visualizar cómo se distribuyeron los fondos en el tiempo.